<a href="https://colab.research.google.com/github/electropavuk/crypto_trader/blob/master/ipynb/system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from enum import Enum

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

np.set_printoptions(edgeitems=10, linewidth=200)
pd.options.mode.chained_assignment = None 

google_drive_dir = '/content/drive/MyDrive/Colab Notebooks/crypto_trader/'
data_dir = google_drive_dir + 'data/act_data/BTCUSDT/'

In [9]:
class Decision(Enum):
    SELL = -1
    WAIT = 0
    BUY = 1

# Trader

In [10]:
class Trader:
    def __init__(self):
        pass

# Experts

In [11]:
class BaseExpert:
    """Base Expert class for decision making."""

    def __init__(self):
        self.name = 'Base Expert'
        self._inner_experts = None
        self._weights = np.ones((len(self._inner_experts), 1))
    
    def estimate(self):
        estimations = np.array([expert.estimate() for expert in self._inner_experts])
        return estimations @ self._weights
    
    def update(self):
        for expert in self._inner_experts:
            expert.update()


class PairExpert(BaseExpert):
    """Expert class for handling specific trading pair.

    Args:
        base: String. Name of base currency.
        quote: String. Name of quote currency.
        history: Dictionary that maps timeframe to historical data (Example: '1h' -> pd.DataFrame).
        experts: List of TimeFrameExpert. Inner experts whose values are considered.
    """

    def __init__(self, base, quote, history, experts):
        self.name = f'{base}/{quote} Expert'
        self._inner_experts = experts



class TimeFrameExpert(BaseExpert):
    """Expert class for handling specific timeframe.

    Args:
        timeframe: String. Name of timeframe (Example: '1h').
        experts: List of RuleExpert. Inner experts whose values are considered.
    """

    def __init__(self, timeframe, experts):
        self.name = f'{timeframe} Expert'
        self._inner_experts = experts

    def construct_data(self, history):
        index = pd.MultiIndex.from_tuples([('History', attr) for attr in history])
        data = pd.DataFrame(history.values, columns=index)
        for expert in self._inner_experts:
            expert.construct_data(data)
        self.data = data
        return self.data

    def get_data(self):
        return self.data



class RuleExpert(BaseExpert):
    """Expert class for handling specific trading rule.

    Args:
        indicator: BaseIndicator. Indicator to which rule is applied.
        rule: BaseRule. Trading rule that applies to indicator.
    """

    def __init__(self, indicator, rule):
        self._indicator = indicator
        self._rule = rule
        self.name = self._indicator.name + self._rule.name

    def construct_data(self, data):
        self._indicator.construct_data(data)

    def estimate(self):
        return self.rule.decide(self.indicator)

    def update(self):
        self.indicator.update()

# Indicators

In [12]:
class BaseIndicator:
    name = 'Base Indicator'

    def __init__(self):
        self._state = None

    def set_name(self):
        return f'{self.name} {self.get_parameters()}'

    def construct_data(self, data):
        raise NotImplementedError

    def get_parameters(self):
        raise NotImplementedError

    def update(self):
        raise NotImplementedError


class MovingAverageIndicator(BaseIndicator):
    name = 'MA'

    def __init__(self, period):
        self.period = period
        self.name = self.set_name()
        self._state = None

    def construct_data(self, data):
        data[self.name, 'MA'] = data['History', 'Close'].rolling(self.period).mean()

    def get_parameters(self):
        return [self.period]

    def update(self):
        raise NotImplementedError

# Rules

In [13]:
class BaseRule:
    name = 'Base Rule'
    
    def __init__(self):
        self._state = None

    def update(self):
        raise NotImplementedError

# Main

In [14]:
def load_history(filename):
    return pd.read_csv(data_dir + filename)

history = load_history('1h.csv')

ind = MovingAverageIndicator(7)
rule = BaseRule()

rule_expert = RuleExpert(ind, rule)
timeframe_expert = TimeFrameExpert('1h', [rule_expert])
timeframe_expert.construct_data(history)

print(timeframe_expert.get_data())

            History                                                                                                                                                                                     MA [7]
          Open time      Open      High       Low     Close       Volume    Close time Quote asset volume Number of trades Taker buy base asset volume Taker buy quote asset volume       Ignore            MA
0      1.502942e+12   4261.48   4313.62   4261.32   4308.83    47.181009  1.502946e+12       2.023661e+05            171.0                   35.160503                 1.509525e+05  7887.635513           NaN
1      1.502946e+12   4308.83   4328.69   4291.37   4315.32    23.234916  1.502950e+12       1.003048e+05            102.0                   21.448071                 9.260828e+04  8039.262402           NaN
2      1.502950e+12   4330.29   4345.45   4309.37   4324.35     7.229691  1.502953e+12       3.128231e+04             36.0                    4.802861                 2.079